In [2]:
import sys
sys.path.append("../")
import numpy as np
import scripts.prediction_utils as predict_u
import scripts.theodolite_utils as theodo_u
import scripts.theodolite_plot_function as theodo_p
import scripts.groundtruth_utils as theodo_g
import scripts.theodolite_function as theodo_f
import importlib
theodo_u = importlib.reload(theodo_u)
theodo_p = importlib.reload(theodo_p)
theodo_f = importlib.reload(theodo_f)
predict_u = importlib.reload(predict_u)

In [9]:
%matplotlib notebook
theodo_u = importlib.reload(theodo_u)
theodo_p = importlib.reload(theodo_p)
predict_u = importlib.reload(predict_u)
theodo_g = importlib.reload(theodo_g)

## Path of main files
path = [
    "../data/20220224/"
]

## Parameters for pre-processing pipeline
file = [
        "/home/maxime/data/ICRA_2023/Vaidis2022_dataset/20220224/20220224_inter_prism.bag"
]

parameters = [1,2,1,1,3,2]  # 1. Apply filtering or not (Module 1), 2-3-4. Parameters tau_r, tau_a, tau_e (Module 1), 5. Parameter tau_s (Module 2), 6 Parameter tau_l (Module 4).

## Range
random_noise_range = [0, 0.004/2, 2]   ## Mean, sigma, ppm,  4mm + 2ppm (2 sigma)  ISO17123-3
## Angles
random_noise_angle = [0, 0.000024241/5*4/2]    # Mean, sigma, 5"=0.000024241 precision datasheet  (2 sigma)  ISO17123-3
## Tilt compensator
random_noise_tilt = [0, 0.000002424/2]    # Mean, sigma, 0.5"=0.000002424 precision datasheet  (2 sigma)  ISO17123-3
## Weather
weather_data_path = "../data/weather_2022/Quebec/"
data_weather = np.array(theodo_u.read_weather_data(weather_data_path+"data_sorted_2022.txt"))
## MC number
num_samples = 1000

## Choice of noise model
model_chosen = [0, 0, 0, 0, 0]  # Tilt noise, Atmospheric correction, extrinsic calibration, Weather corrections, Time synchronization # 1: Activate, 0:Disable

../data/weather_2022/Quebec/data_sorted_2022.txt


In [15]:
predict_u = importlib.reload(predict_u)
for path_i, rosbag_i in zip(path, file):
    print(file)

    ## Number of rigid transforms to apply the uncertainty
    path_file_GCP = "total_stations/GCP.txt"
    frame_chosen, T_1_corrected, T_2_corrected, T_3_corrected = theodo_g.extrinsic_calibration_noise(path_i + path_file_GCP, random_noise_range, random_noise_angle, random_noise_tilt, num_samples)
    ## Read Extrinsic results for RTS
    ## Estimated extrinsic calibration uncertainty
    _, _, _, Tf_1, Tf_2, Tf_3 = theodo_u.read_marker_file(path_i + path_file_GCP, 1, 1)
    # Reading sensor extrinsic calibration
    file_sensors = theodo_u.if_file_exist(path_i + "sensors_extrinsic_calibration/calibration_results.csv",'')
    extrinsic_calibration_results = theodo_u.read_extrinsic_calibration_results_file(file_sensors)

    thresold_d = parameters[1]  ## tau_r [m/s]
    thresold_a = parameters[2]  ## tau_a [deg/s]
    thresold_e = parameters[3]  ## tau_e [deg/s]
    limit_time_interval = parameters[4]  ## tau_s [s]

    t1, t2, t3, _, _, _, d1, d2, d3, a1, a2, a3, e1, e2, e3 = theodo_u.read_rosbag_theodolite_without_tf_raw_data_pre_filtered(rosbag_i)
    index_1_f = theodo_u.thresold_raw_data(t1, d1, a1, e1, thresold_d, thresold_a * 3.1415926 / 180,
                                           thresold_e * 3.1415926 / 180, limit_time_interval)
    index_2_f = theodo_u.thresold_raw_data(t2, d2, a2, e2, thresold_d, thresold_a * 3.1415926 / 180,
                                           thresold_e * 3.1415926 / 180, limit_time_interval)
    index_3_f = theodo_u.thresold_raw_data(t3, d3, a3, e3, thresold_d, thresold_a * 3.1415926 / 180,
                                           thresold_e * 3.1415926 / 180, limit_time_interval)
    T1, D1, A1, E1 = predict_u.data_L_Raw_data(t1, d1, a1, e1, index_1_f)
    T2, D2, A2, E2 = predict_u.data_L_Raw_data(t2, d2, a2, e2, index_2_f)
    T3, D3, A3, E3 = predict_u.data_L_Raw_data(t3, d3, a3, e3, index_3_f)
    print(len(T1), len(T2), len(T3))




['/home/maxime/data/ICRA_2023/Vaidis2022_dataset/20220224/20220224_inter_prism.bag']
Number of data for theodolites: [4945 4681 4737]
Bad measures: 558
3836 3619 3690


In [13]:
print(t1[index_1_f])

[1.64573488e+09 1.64573488e+09 1.64573488e+09 ... 1.64573735e+09
 1.64573735e+09 1.64573735e+09]
